# Exercise 4: DCM for fMRI

## Exercise 4.1: Programming a two region DCM

a. writing a function to calculate the right-hand side of the differential equation (1)

In [12]:
function single_step_neural(x::Vector{Float64}, u::Vector{Float64}, Pneural::Dict{String, Matrix{Float64}})
    u1, u2 = u[1], u[2] # u1 is driving input, u2 is modulating input of 2d input vector u (u1 and u2 are columns)
    A = Pneural["A"]    # fixed connectivity matrix
    B = Pneural["B"]    # modulatory connectivity matrix
    C = Pneural["C"]    # driving input matrix??

    # defining connectivity matrix CON
    CON = zeros(2, 2)
    CON[1, 1] = -0.5 * exp(A[1, 1] + u2 * B[1, 1])
    CON[1, 2] = A[1, 2] + u2 * B[1, 2]
    CON[2, 1] = A[2, 1] + u2 * B[2, 1]
    CON[2, 2] = -0.5 * exp(A[2, 2] + u2 * B[2, 2])

    # computing dx/dt
    dx_dt = CON * x + C * [u1, 0.0]
    return dx_dt
end

single_step_neural (generic function with 1 method)

In [13]:
# testing 4.1a (this was all chatGPT tbh... but supposedly its gives the expected output)
using LinearAlgebra, Test

A = [0.1 0.2; 0.3 0.4]

B = [0.01 0.02; 0.03 0.04]

C = [1.0 0.0; 0.0 0.0]

Pneural = Dict("A" => A, "B" => B, "C" => C)

x = [1.0, 2.0]
u = [5.0, 1.0]

dx_dt = single_step_neural(x, u, Pneural)
println("dx/dt = ", dx_dt)

dx/dt = [4.881860964770564, -1.222707218511336]


b.  writing a function that computes the update for the hemodynamic model of a single region

In [14]:
function single_step_hrf(h::Vector{Float64}, x::Vector{Float64}, P_HRF::Dict{String, Float64})
    s, f, v, q = h

    # note: in this exercise, fixed values: κ = 0.64, γ = 0.32, τ = 2, α = 0.32 and E0 = 0.4
    k = PHRF["κ"]
    y = PHRF["γ"]
    t = PHRF["τ"]
    a = PHRF["α"]
    E0 = PHRF["E0"]

    # getting ds (eq. 4)
    ds = x - k * s - y * (f - 1)
    # getting df (eq. 5)
    df = s
    # getting dv (eq. 6)
    dv = (1 / t) * (f - v^(1 / a))
    # getting dv (eq. 6)
    interm = (1 - (1 - E0)^(1 / f)) / E0
    dq = (1/t) * (f * interm - v^(1 / a) * (q / v))

    return [ds, df, dv, dq]
end

single_step_hrf (generic function with 1 method)